In [4]:
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.builder.appName("tip_count_sql").getOrCreate()

22/12/25 21:29:33 WARN Utils: Your hostname, PSui-MacBookPro.local resolves to a loopback address: 127.0.0.1; using 192.168.0.7 instead (on interface en0)
22/12/25 21:29:33 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/25 21:29:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/12/25 21:29:33 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/12/25 21:29:33 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


# row 데이터 생성
1. wget -i https://d37ci6vzurychx.cloudfront.net/trip-data/fhv_tripdata_2020-03.parquet -P ../../../../data/ -w 2
    1. (전체 데이터가 필요한 경우) ./install_row_data.sh
2. !conda install pyarrow fastparquet -y
3. convert parquet to csv

In [34]:
!conda install pyarrow fastparquet -y

done
Solving environment: done

## Package Plan ##

  environment location: /usr/local/anaconda3

  added / updated specs:
    - fastparquet
    - pyarrow


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    abseil-cpp-20211102.0      |       he9d5cce_0         933 KB
    arrow-cpp-8.0.0            |   py39had1886b_0         7.0 MB
    aws-c-common-0.4.57        |       hb1e8313_1         140 KB
    aws-c-event-stream-0.1.6   |       h23ab428_5          21 KB
    aws-checksums-0.1.9        |       hb1e8313_0          47 KB
    aws-sdk-cpp-1.8.185        |       he271ece_0         1.6 MB
    boost-cpp-1.73.0           |      hca72f7f_12          17 KB
    conda-22.11.1              |   py39hecd8cb5_4         941 KB
    fastparquet-0.5.0          |   py39h67323c0_2         173 KB
    gflags-2.2.2               |       h0a44026_0         104 KB
    glog-0.5.0                 |       h23ab428_0

In [8]:
import pandas as pd
# !pip install pyarrow fastparquet

target_directory = '../../../../data'
trip_file = 'fhvhv_tripdata_2020-03'
zone_file = '01/taxi+_zone_lookup'  # 01 추가

In [53]:
trip_df = pd.read_parquet('%s/%s.parquet' % (target_directory, trip_file))
trip_df.to_csv(f'{target_directory}/{trip_file}.csv')

In [9]:
trip_data = spark.read.csv(f'{target_directory}/{trip_file}.csv', header=True)
zone_data = spark.read.csv(f'{target_directory}/{zone_file}.csv', header=True)

`header=True`

아래와 같은 형식 생성을 방지할 수 있음.
+----+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+----+----------+-------------------+-----------------+------------------+----------------+--------------+
| _c0|              _c1|                 _c2|                 _c3|                _c4|                _c5|                _c6|                _c7|         _c8|         _c9|      _c10|     _c11|               _c12| _c13|_c14|     _c15|                _c16|       _c17|_c18|      _c19|               _c20|             _c21|              _c22|            _c23|          _c24|
+----+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+----+----------+-------------------+-----------------+------------------+----------------+--------------+
|null|hvfhs_license_num|dispatching_base_num|originating_base_num|   request_datetime|  on_scene_datetime|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|trip_miles|trip_time|base_passenger_fare|tolls| bcf|sales_tax|congestion_surcharge|airport_fee|tips|driver_pay|shared_request_flag|shared_match_flag|access_a_ride_flag|wav_request_flag|wav_match_flag|
|   0|           HV0005|              B02510|                null|2020-03-01 00:00:12|               null|2020-03-01 00:03:40|2020-03-01 00:23:39|          81|         159|     8.655|     1199|              24.45|  0.0|0.54|      1.9|                 0.0|       null| 0.0|     19.65|                  N|                N|                 N|               N|             N|


In [57]:
trip_data.show()

22/12/25 21:08:31 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , hvfhs_license_num, dispatching_base_num, originating_base_num, request_datetime, on_scene_datetime, pickup_datetime, dropoff_datetime, PULocationID, DOLocationID, trip_miles, trip_time, base_passenger_fare, tolls, bcf, sales_tax, congestion_surcharge, airport_fee, tips, driver_pay, shared_request_flag, shared_match_flag, access_a_ride_flag, wav_request_flag, wav_match_flag
 Schema: _c0, hvfhs_license_num, dispatching_base_num, originating_base_num, request_datetime, on_scene_datetime, pickup_datetime, dropoff_datetime, PULocationID, DOLocationID, trip_miles, trip_time, base_passenger_fare, tolls, bcf, sales_tax, congestion_surcharge, airport_fee, tips, driver_pay, shared_request_flag, shared_match_flag, access_a_ride_flag, wav_request_flag, wav_match_flag
Expected: _c0 but found: 
CSV file: file:///Users/auto/github/spark/data/fhvhv_tripdata_2020-03.csv
+---+-----------------+----------------

In [14]:
trip_data.createOrReplaceTempView("trip_data")
zone_data.createOrReplaceTempView("zone_data")

In [15]:
spark.sql("select * from trip_data").limit(5).show()

22/12/25 21:35:02 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , hvfhs_license_num, dispatching_base_num, originating_base_num, request_datetime, on_scene_datetime, pickup_datetime, dropoff_datetime, PULocationID, DOLocationID, trip_miles, trip_time, base_passenger_fare, tolls, bcf, sales_tax, congestion_surcharge, airport_fee, tips, driver_pay, shared_request_flag, shared_match_flag, access_a_ride_flag, wav_request_flag, wav_match_flag
 Schema: _c0, hvfhs_license_num, dispatching_base_num, originating_base_num, request_datetime, on_scene_datetime, pickup_datetime, dropoff_datetime, PULocationID, DOLocationID, trip_miles, trip_time, base_passenger_fare, tolls, bcf, sales_tax, congestion_surcharge, airport_fee, tips, driver_pay, shared_request_flag, shared_match_flag, access_a_ride_flag, wav_request_flag, wav_match_flag
Expected: _c0 but found: 
CSV file: file:///Users/auto/github/spark/data/fhvhv_tripdata_2020-03.csv
+---+-----------------+----------------

In [16]:
spark.sql("select * from zone_data").limit(5).show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
+----------+-------------+--------------------+------------+



In [35]:
# PULocation Pickup location
# 멘하탄 승차
spark.sql("select borough, count(*) as tiprs "
          "from (select zone_data.Borough as borough"
                " from trip_data join zone_data on trip_data.PULocationID = zone_data.LocationID) group by borough order by tiprs desc").show()

+-------------+-------+
|      borough|  tiprs|
+-------------+-------+
|    Manhattan|4953147|
|     Brooklyn|3735765|
|       Queens|2437394|
|        Bronx|2086597|
|Staten Island| 178818|
|      Unknown|    845|
|          EWR|    362|
+-------------+-------+



In [36]:
# DOLocation Drop location
# 멘하탄 하차, EWR, 공항이 증가함
spark.sql("select borough, count(*) as tiprs "
          "from (select zone_data.Borough as borough"
                " from trip_data join zone_data on trip_data.DOLocationID = zone_data.LocationID) group by borough order by tiprs desc").show()

+-------------+-------+
|      borough|  tiprs|
+-------------+-------+
|    Manhattan|4553783|
|     Brooklyn|3696684|
|       Queens|2468416|
|        Bronx|2043492|
|      Unknown| 387760|
|Staten Island| 177727|
|          EWR|  65066|
+-------------+-------+



# explain(True)
## 실행 계획 확인하기

In [37]:
spark.sql("select borough, count(*) as tiprs "
          "from (select zone_data.Borough as borough"
                " from trip_data join zone_data on trip_data.DOLocationID = zone_data.LocationID) group by borough order by tiprs desc").explain(True)

== Parsed Logical Plan ==
'Sort ['tiprs DESC NULLS LAST], true
+- 'Aggregate ['borough], ['borough, 'count(1) AS tiprs#1138]
   +- 'SubqueryAlias __auto_generated_subquery_name
      +- 'Project ['zone_data.Borough AS borough#1137]
         +- 'Join Inner, ('trip_data.DOLocationID = 'zone_data.LocationID)
            :- 'UnresolvedRelation [trip_data], [], false
            +- 'UnresolvedRelation [zone_data], [], false

== Analyzed Logical Plan ==
borough: string, tiprs: bigint
Sort [tiprs#1138L DESC NULLS LAST], true
+- Aggregate [borough#1137], [borough#1137, count(1) AS tiprs#1138L]
   +- SubqueryAlias __auto_generated_subquery_name
      +- Project [Borough#152 AS borough#1137]
         +- Join Inner, (DOLocationID#93 = LocationID#151)
            :- SubqueryAlias trip_data
            :  +- View (`trip_data`, [_c0#84,hvfhs_license_num#85,dispatching_base_num#86,originating_base_num#87,request_datetime#88,on_scene_datetime#89,pickup_datetime#90,dropoff_datetime#91,PULocationID#92,D